In [6]:
!pip install h5py

     |████████████████████████████████| 4.5 MB 3.3 MB/s eta 0:00:01


In [3]:
!pip install opencv-python

     |████████████████████████████████| 60.4 MB 9.9 MB/s eta 0:00:01


In [7]:
import os
import cv2
import datetime
import numpy as np
import h5py

In [12]:
print(os.getcwd())

path = '/home/vividlee/Desktop/YeloCar/Data/'

files = os.listdir(path)
print(len(files))

/home/vividlee/Desktop/YeloCar
688


In [16]:
print(files)

['20211228_105148.mp4', '20211228_155236.log', '20211217_180009.mp4', '20211228_162436.log', '20211216_170453.log', '20211222_111048.log', '20211216_185247.mp4', '20211227_143834.mp4', '20211216_170702.log', '20211217_175556.log', '20211228_160541.log', '20211222_110537.log', '20211227_160643.log', '20211228_110316.mp4', '20211227_155111.log', '20211227_153616.mp4', '20211227_155631.log', '20211217_180352.log', '20211228_112107.mp4', '20211228_160236.mp4', '20211227_135516.mp4', '20211227_143004.log', '20211227_152233.log', '20211217_162323.log', '20211227_142234.mp4', '20211227_160953.mp4', '20211228_162637.log', '20211227_160312.mp4', '20211217_161059.log', '20211222_113111.log', '20211217_163352.mp4', '20211228_104755.log', '20211227_135631.mp4', '20211228_163715.log', '20211217_180449.log', '20211228_163035.log', '20211217_180637.mp4', '20211228_155734.mp4', '20211228_162110.log', '20211216_171854.mp4', '20211228_163317.log', '20211227_154637.mp4', '20211227_153736.log', '20211222_

In [17]:
sorted_files = sorted(files)
print(sorted_files)

['20211210_164307.mp4', '20211210_164317.log', '20211210_164422.mp4', '20211210_164432.log', '20211210_164539.mp4', '20211210_164548.log', '20211210_164649.mp4', '20211210_164659.log', '20211210_164803.mp4', '20211210_164813.log', '20211210_164930.mp4', '20211210_164940.log', '20211210_165041.mp4', '20211210_165050.log', '20211210_165149.mp4', '20211210_165158.log', '20211210_165300.mp4', '20211210_165309.log', '20211210_165402.mp4', '20211210_165411.log', '20211210_165512.mp4', '20211210_165521.log', '20211210_165624.mp4', '20211210_165634.log', '20211210_165734.mp4', '20211210_165743.log', '20211210_165837.mp4', '20211210_165846.log', '20211210_165951.mp4', '20211210_170001.log', '20211210_170100.mp4', '20211210_170109.log', '20211210_170208.mp4', '20211210_170217.log', '20211210_170311.mp4', '20211210_170320.log', '20211210_170410.mp4', '20211210_170419.log', '20211210_170510.mp4', '20211210_170519.log', '20211210_170615.mp4', '20211210_170625.log', '20211210_170725.mp4', '20211210_

In [18]:
# Dataset 구성
# Data : ALL
# fps : 2/41
# cv.imshow : image

test_name = 'deep4pc_test1'
  
image_list = []
action_list = []

hsv_lower = np.array([20, 90, 123])
hsv_upper = np.array([50, 255, 255])

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
       
def make_dataset(path, filename_avi, filename_log):
    print("filename_avi : {}, filename_log : {}".format(filename_avi, filename_log))
    image_list = []
    action_list = []

    logpath = path + filename_log + '.log'
    avipath = path + filename_avi + '.mp4'

    logfile = open(logpath, 'r')
    avifile = cv2.VideoCapture(avipath)

    loglines = logfile.readlines()

    current_avi_time = 0
    start_time = datetime.datetime(int(filename_avi[0:4]), int(filename_avi[4:6]), int(filename_avi[6:8]), int(filename_avi[9:11]), int(filename_avi[11:13]), int(filename_avi[13:15]))

    i = 0
    for line in loglines[1:]:
        i += 1
        logdata = line.split(' ')
        date = logdata[0].split('-')
        time = logdata[1].split(':')
        log_time = datetime.datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1]), int(time[2].split('.')[0]), int(time[2].split('.')[1]))

        delt_time = (log_time-start_time).total_seconds()

        image = None
       
        # steer = (int(logdata[4].split('/')[0]) - 1108) / (1888 - 1108)
        # throttle = (int(logdata[7].split('/')[0]) - 1352) / (1840 - 1352)
        steer = (int(logdata[2]) - 1100) / (1900 - 1100)          # max_steeringValue : 1896, min_steeringValue : 1104
        throttle = (int(logdata[3]) - 1350) / (1760 - 1350)       # max_throttleValue : 1720, min_throttleValue : 1356
       
        while(avifile.isOpened() and current_avi_time < delt_time):
            ret, frame = avifile.read()
            if ret == True:
                current_avi_time += 2/41
                if current_avi_time >= delt_time:
                    if i % 100 == 1 :
                        print("current_avi_time : {:.6f} > delt_time : {:.6f}".format(current_avi_time, delt_time))
                        print("line : {}, logdata[2] : {}, logdata[3] : {}, steer : {}, throttle : {:.6f}".format(i, logdata[2], logdata[3], steer, throttle))
                    image = cv2.resize(frame, (64, 48))

                    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                    mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
                    masked = cv2.bitwise_and(image, image, mask=mask)
                    #cv2.imshow('frame', image)
                    #cv2.waitKey(1)

                    image_list.append(masked)
                    action_list.append([steer, throttle])
            else:
                print("capture failed with line : {}".format(i))
                print("current_avi_time : {:.6f} > delt_time : {:.6f}".format(current_avi_time, delt_time))
                print("line : {}, logdata[2] : {}, logdata[3] : {}, steer : {}, throttle : {:.6f}".format(i, logdata[2], logdata[3], steer, throttle))
                break

        if i % 1000 == 0:
            print(i)
           
    print('read finished')

    avifile.release()
    cv2.destroyAllWindows()
    logfile.close()

    print('make temp dataset')
    image_train = np.array(image_list[:int(len(image_list)*0.8)])
    action_train = np.array(action_list[:int(len(image_list)*0.8)])
    image_val = np.array(image_list[int(len(image_list)*0.8):])
    action_val = np.array(action_list[int(len(image_list)*0.8):])

    train_len = len(image_train)
    val_len = len(image_val)

    hdf5_directory = test_name
    createFolder('./HDF5/' + hdf5_directory)
   
    with h5py.File('./HDF5/'+ hdf5_directory + '/' + filename_avi +'_low_temp.hdf5', 'w') as hf :
        print("write in ./HDF5/" + hdf5_directory + "/" + filename_avi + "_low_temp.hdf5\n")
        dset_x_train = hf.create_dataset('x_train', (train_len, 48, 64, 3), chunks=True, compression="gzip")
        dset_y_train = hf.create_dataset('y_train', (train_len, 2), chunks=True, compression="gzip")
        dset_x_val = hf.create_dataset('x_val', (val_len, 48, 64, 3), chunks=True, compression="gzip")
        dset_y_val = hf.create_dataset('y_val', (val_len, 2), chunks=True, compression="gzip")

        dset_x_train[:, :, :, :] = image_train
        dset_y_train[:, :] = action_train
        dset_x_val[:, :, :, :] = image_val
        dset_y_val[:, :] = action_val

    print(filename_avi + ' finished\n')
    
    
if __name__ == '__main__':
    
    path = '/home/vividlee/Desktop/YeloCar/Data/'

    files = os.listdir(path)
    sorted_files = sorted(files)
    for i in range(0, len(sorted_files), 2) :
        make_dataset(path, sorted_files[i].split('.')[0], sorted_files[i+1].split('.')[0])

    image_train = None
    action_train = None
    image_val = None
    action_val = None
   
    for i in range(0, len(files), 2):
        filename = sorted_files[i].split('.')[0]
        # print("In main, filename : {}".format(filename))
        filename = './HDF5/'+ test_name + '/' + filename + '_low_temp.hdf5'
       
        with h5py.File(filename, 'r') as hf:
            print(filename)
            if image_train is None:
                image_train = np.array(hf['x_train'][:, :, :, :])
                action_train = np.array(hf['y_train'][:, :])
                image_val = np.array(hf['x_val'][:, :, :, :])
                action_val = np.array(hf['y_val'][:, :])
            else:
                image_train = np.concatenate((image_train, hf['x_train']), axis=0)
                action_train = np.concatenate((action_train, hf['y_train']), axis=0)
                image_val = np.concatenate((image_val, hf['x_val']), axis=0)
                action_val = np.concatenate((action_val, hf['y_val']), axis=0)

    train_len = len(image_train)
    val_len = len(image_val)
 
    model_directory = 'Model_' + test_name
    createFolder('./Model/' + model_directory)
 
    with h5py.File('./Model/' + model_directory + '/dataset_hsv_' + test_name + '.hdf5', 'w') as hf:
        print("write in /Model/"  + model_directory + "/dataset_hsv_" + test_name + ".hdf5\n")
        dset_x_train = hf.create_dataset('x_train', (train_len, 48, 64, 3), chunks=True, compression="gzip")
        dset_y_train = hf.create_dataset('y_train', (train_len, 2), chunks=True, compression="gzip")
        dset_x_val = hf.create_dataset('x_val', (val_len, 48, 64, 3), chunks=True, compression="gzip")
        dset_y_val = hf.create_dataset('y_val', (val_len, 2), chunks=True, compression="gzip")

        dset_x_train[:, :, :, :] = image_train
        dset_y_train[:, :] = action_train
        dset_x_val[:, :, :, :] = image_val
        dset_y_val[:, :] = action_val

    # dataset = tf.data.Dataset.from_tensors((image_feature[i], action_feature[i]) for i in range(len(image_feature)))
    # tf.data.experimental.save(dataset, path+'saved_data')

filename_avi : 20211210_164307, filename_log : 20211210_164317
current_avi_time : 12.048780 > delt_time : 12.000151
line : 1, logdata[2] : 1500, logdata[3] : 1364
, steer : 0.5, throttle : 0.034146
current_avi_time : 18.292683 > delt_time : 18.266052
line : 101, logdata[2] : 1500, logdata[3] : 1424
, steer : 0.5, throttle : 0.180488
current_avi_time : 37.024390 > delt_time : 36.978458
line : 401, logdata[2] : 1384, logdata[3] : 1436
, steer : 0.355, throttle : 0.209756
current_avi_time : 43.268293 > delt_time : 43.220479
line : 501, logdata[2] : 1480, logdata[3] : 1432
, steer : 0.475, throttle : 0.200000
current_avi_time : 49.463415 > delt_time : 49.440884
line : 601, logdata[2] : 1496, logdata[3] : 1436
, steer : 0.495, throttle : 0.209756
current_avi_time : 55.707317 > delt_time : 55.687265
line : 701, logdata[2] : 1500, logdata[3] : 1364
, steer : 0.5, throttle : 0.034146
read finished
make temp dataset
write in ./HDF5/deep4pc_test1/20211210_164307_low_temp.hdf5

20211210_164307 fi

current_avi_time : 36.390244 > delt_time : 36.362506
line : 401, logdata[2] : 1500, logdata[3] : 1440
, steer : 0.5, throttle : 0.219512
current_avi_time : 42.585366 > delt_time : 42.573577
line : 501, logdata[2] : 1432, logdata[3] : 1440
, steer : 0.415, throttle : 0.219512
current_avi_time : 48.878049 > delt_time : 48.836028
line : 601, logdata[2] : 1500, logdata[3] : 1360
, steer : 0.5, throttle : 0.024390
read finished
make temp dataset
write in ./HDF5/deep4pc_test1/20211210_165041_low_temp.hdf5

20211210_165041 finished

filename_avi : 20211210_165149, filename_log : 20211210_165158
current_avi_time : 11.219512 > delt_time : 11.218835
line : 1, logdata[2] : 1500, logdata[3] : 1360
, steer : 0.5, throttle : 0.024390
current_avi_time : 30.000000 > delt_time : 29.962691
line : 301, logdata[2] : 1444, logdata[3] : 1436
, steer : 0.43, throttle : 0.209756
current_avi_time : 36.243902 > delt_time : 36.195176
line : 401, logdata[2] : 1500, logdata[3] : 1436
, steer : 0.5, throttle : 0.20

20211210_165837 finished

filename_avi : 20211210_165951, filename_log : 20211210_170001
current_avi_time : 11.804878 > delt_time : 11.785982
line : 1, logdata[2] : 1504, logdata[3] : 1364
, steer : 0.505, throttle : 0.034146
current_avi_time : 18.048780 > delt_time : 18.024694
line : 101, logdata[2] : 1500, logdata[3] : 1448
, steer : 0.5, throttle : 0.239024
current_avi_time : 24.292683 > delt_time : 24.263156
line : 201, logdata[2] : 1356, logdata[3] : 1444
, steer : 0.32, throttle : 0.229268
current_avi_time : 30.536585 > delt_time : 30.530697
line : 301, logdata[2] : 1500, logdata[3] : 1456
, steer : 0.5, throttle : 0.258537
current_avi_time : 36.780488 > delt_time : 36.768248
line : 401, logdata[2] : 1500, logdata[3] : 1456
, steer : 0.5, throttle : 0.258537
current_avi_time : 43.024390 > delt_time : 43.014833
line : 501, logdata[2] : 1500, logdata[3] : 1456
, steer : 0.5, throttle : 0.258537
current_avi_time : 49.268293 > delt_time : 49.247498
line : 601, logdata[2] : 1552, logd

current_avi_time : 36.390244 > delt_time : 36.379337
line : 401, logdata[2] : 1828, logdata[3] : 1460
, steer : 0.91, throttle : 0.268293
current_avi_time : 42.634146 > delt_time : 42.611992
line : 501, logdata[2] : 1504, logdata[3] : 1464
, steer : 0.505, throttle : 0.278049
current_avi_time : 48.878049 > delt_time : 48.853398
line : 601, logdata[2] : 1504, logdata[3] : 1360
, steer : 0.505, throttle : 0.024390
read finished
make temp dataset
write in ./HDF5/deep4pc_test1/20211210_170725_low_temp.hdf5

20211210_170725 finished

filename_avi : 20211210_170830, filename_log : 20211210_170840
current_avi_time : 11.951220 > delt_time : 11.950249
line : 1, logdata[2] : 1496, logdata[3] : 1392
, steer : 0.495, throttle : 0.102439
current_avi_time : 18.243902 > delt_time : 18.205568
line : 101, logdata[2] : 1208, logdata[3] : 1468
, steer : 0.135, throttle : 0.287805
current_avi_time : 30.731707 > delt_time : 30.694555
line : 301, logdata[2] : 1500, logdata[3] : 1480
, steer : 0.5, throttle 

current_avi_time : 11.560976 > delt_time : 11.542396
line : 1, logdata[2] : 1496, logdata[3] : 1360
, steer : 0.495, throttle : 0.024390
current_avi_time : 17.804878 > delt_time : 17.778501
line : 101, logdata[2] : 1120, logdata[3] : 1464
, steer : 0.025, throttle : 0.278049
current_avi_time : 24.048780 > delt_time : 24.024303
line : 201, logdata[2] : 1424, logdata[3] : 1464
, steer : 0.405, throttle : 0.278049
current_avi_time : 30.292683 > delt_time : 30.261827
line : 301, logdata[2] : 1500, logdata[3] : 1468
, steer : 0.5, throttle : 0.287805
current_avi_time : 36.536585 > delt_time : 36.500052
line : 401, logdata[2] : 1496, logdata[3] : 1464
, steer : 0.495, throttle : 0.278049
current_avi_time : 42.780488 > delt_time : 42.740255
line : 501, logdata[2] : 1496, logdata[3] : 1368
, steer : 0.495, throttle : 0.043902
read finished
make temp dataset
write in ./HDF5/deep4pc_test1/20211216_170001_low_temp.hdf5

20211216_170001 finished

filename_avi : 20211216_170055, filename_log : 2021

...

20211228_163706 finished

filename_avi : 20211228_163851, filename_log : 20211228_163901
current_avi_time : 11.804878 > delt_time : 11.761218
line : 1, logdata[2] : 1500, logdata[3] : 1360
, steer : 0.5, throttle : 0.024390
current_avi_time : 18.048780 > delt_time : 18.011225
line : 101, logdata[2] : 1472, logdata[3] : 1432
, steer : 0.465, throttle : 0.200000
current_avi_time : 24.292683 > delt_time : 24.257888
line : 201, logdata[2] : 1436, logdata[3] : 1436
, steer : 0.42, throttle : 0.209756
current_avi_time : 30.536585 > delt_time : 30.495126
line : 301, logdata[2] : 1432, logdata[3] : 1448
, steer : 0.415, throttle : 0.239024
current_avi_time : 36.731707 > delt_time : 36.722898
line : 401, logdata[2] : 1556, logdata[3] : 1440
, steer : 0.57, throttle : 0.219512
current_avi_time : 42.975610 > delt_time : 42.974112
line : 501, logdata[2] : 1640, logdata[3] : 1444
, steer : 0.675, throttle : 0.229268
capture failed with line : 523
current_avi_time : 44.292683 > delt_time : 44.346857

capture failed with line : 640
current_avi_time : 44.292683 > delt_time : 51.649224
line : 640, logdata[2] : 1496, logdata[3] : 1364
, steer : 0.495, throttle : 0.034146
capture failed with line : 641
current_avi_time : 44.292683 > delt_time : 51.714650
line : 641, logdata[2] : 1504, logdata[3] : 1364
, steer : 0.505, throttle : 0.034146
capture failed with line : 642
current_avi_time : 44.292683 > delt_time : 51.782066
line : 642, logdata[2] : 1500, logdata[3] : 1364
, steer : 0.5, throttle : 0.034146
capture failed with line : 643
current_avi_time : 44.292683 > delt_time : 51.849350
line : 643, logdata[2] : 1500, logdata[3] : 1364
, steer : 0.5, throttle : 0.034146
capture failed with line : 644
current_avi_time : 44.292683 > delt_time : 51.885106
line : 644, logdata[2] : 1496, logdata[3] : 1364
, steer : 0.495, throttle : 0.034146
capture failed with line : 645
current_avi_time : 44.292683 > delt_time : 51.952477
line : 645, logdata[2] : 1500, logdata[3] : 1360
, steer : 0.5, thrott

20211228_163950 finished

./HDF5/deep4pc_test1/20211210_164307_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_164422_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_164539_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_164649_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_164803_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_164930_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165041_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165149_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165300_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165402_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165512_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165624_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165734_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165837_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_165951_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_170100_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_170208_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_170311_low_temp.hdf5
./HDF5/deep4pc_test1/20211210_170410_low_temp.hdf5
./HDF

./HDF5/deep4pc_test1/20211222_113513_low_temp.hdf5
./HDF5/deep4pc_test1/20211222_113614_low_temp.hdf5
./HDF5/deep4pc_test1/20211222_113715_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_134903_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_135129_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_135245_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_135404_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_135516_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_135631_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_135925_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140040_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140157_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140300_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140416_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140526_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140634_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_140820_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_141049_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_141151_low_temp.hdf5
./HDF5/deep4pc_test1/20211227_1

./HDF5/deep4pc_test1/20211228_161723_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_161821_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_161919_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162012_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162101_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162200_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162251_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162427_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162529_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162628_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162734_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_162833_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163026_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163121_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163218_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163307_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163410_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163507_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_163613_low_temp.hdf5
./HDF5/deep4pc_test1/20211228_1